  
[**Dr. Kevin M. Moerman**](mailto:kevin.moerman@nuigalway.ie), _Lecturer in Biomedical Engineering_   

National University of Ireland Galway.   

---

\newpage

# The truss element
![spring](truss1.png)  

* Behaviour Hooke's law for a bar:    
$$\sigma=E\epsilon=\frac{F}{A}$$     
$$\begin{Bmatrix} f_{1} \\ f_{2} \end{Bmatrix}=\frac{AE}{L}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}\begin{Bmatrix} u_1 \\ u_2\end{Bmatrix}$$   

* Truss (or bar) is a 1D element but has a **cross-sectional area** $A$    

* **strain** a relative metric for deformation: 
$$\epsilon=\frac{(u_2-u_1)}{L_0}$$    
  
$\sigma$: Stress (Pa or $N/m^2$)   
$F$: Force (N)   
$E$: The material Young's Modulus (Pa or $N/m^2$)   
$\epsilon$: The strain in the material ([])   
$L$ truss lenght (m)

## Example: A two truss system and a known force
Consider the two truss system below. 

* The system consists of 2 truss elements and 3 nodes.   

* Node 1 is contrained from moving.   

* A force of 10 N is applied to node 2 in the positive x-direction.   

$E^{(1)}=200$ $MPa$   
$L^{(1)}=100$ $mm$   
$A^{(1)}=20$ $mm^2$   
   
$E^{(2)}=200$ $MPa$   
$L^{(2)}=100$ $mm$  
$A^{(2)}=10$ $mm^2$   


![](trussSet_2_force.png)   

**Assignment**: Determine the nodal displacements, reaction forces and strains within each element of the bar.         


### Boundary and compatibility conditions
* The boundary conditions: 
$$u_1=0$$
$$F_3=P=10$$
  
    
![](trussSet_2_force.png)  

### Setting up the element stiffness matrices
* Recognize: 
$$K^{(i)}=\frac{A^{(i)}E^{(i)}}{L^{(i)}}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}=k^{(i)}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}$$   
* Derive individual stiffness values   
$$k^{(1)}=\frac{A^{(1)}E^{(1)}}{L^{(1)}}=\frac{20*200\times10^3}{100}=40$$
$$k^{(2)}=\frac{A^{(2)}E^{(2)}}{L^{(2)}}=\frac{10*200\times10^3}{100}=20$$
* Derive individual stiffness matrices   
$$K^{(1)}=\begin{bmatrix} 40 & -40 \\ -40 & 40\end{bmatrix}$$   
  
$$K^{(2)}=\begin{bmatrix} 20 & -20 \\ -20 & 20\end{bmatrix}$$


In [7]:
I=[1 -1; -1 1]; % Array for spawning stiffness matrices
k1=(20*200)/100; % Stiffness 1
k2=(10*200)/100; % Stiffness 2
K1=k1*I % Element stiffness matrix 1
K2=k2*I % Element stiffness matrix 2

K1 =

   40  -40
  -40   40

K2 =

   20  -20
  -20   20



### Superposition to assemble global stiffness matrix
Through superposition to global stiffness matrix can be assembled. The superposition is often written as:
$$K=\sum_{i=1}^{N} K^{(i)}$$
However, it should be noted this is not a summation. The below numerical implementation illustrates how **the indices of the nodes involved are used as indices into matrix $K$**, leading to:
$$\begin{bmatrix} K \end{bmatrix}=\begin{bmatrix} 40 & -40 & 0 \\ -40 & 60 & -20 \\ 0 & -20 & 20\end{bmatrix}$$


In [8]:
K=zeros(3,3); %Initialize stiffness aray with all zeros
K([1,2],[1,2])= K([1,2],[1,2])+ K1; %Add element 1 contribution
K([2,3],[2,3])= K([2,3],[2,3])+ K2; %Add element 2 contribution
K

K =

   40  -40    0
  -40   60  -20
    0  -20   20



### Solving the system using matrix algebra
* Following derivation of the global stiffness matrix, and using $u_1=0$, $F_2=0$, $F_3=P=10$, the total system now becomes:
$$\begin{Bmatrix} F_1 \\ 0 \\ 10 \end{Bmatrix}=\begin{bmatrix} 40 & -40 & 0 \\ -40 & 60 & -20 \\ 0 & -20 & 20\end{bmatrix}\begin{Bmatrix} 0 \\ u_2 \\ u_3\end{Bmatrix}$$
* Select the 2-3 sub-system: 
$$\begin{Bmatrix} 0 \\ 10 \end{Bmatrix}=\begin{bmatrix} 60 & -20 \\ -20 & 20\end{bmatrix}\begin{Bmatrix} u_2 \\ u_3\end{Bmatrix}$$

* Solve: 
$$\begin{Bmatrix} u_2 \\ u_3\end{Bmatrix}=\begin{bmatrix} 60 & -20 \\ -20 & 20\end{bmatrix}^{-1}\begin{Bmatrix} 0 \\ 10 \end{Bmatrix}$$

In [9]:
F23=[0 10]'; %Force at node 2 and 3
K23=K([2,3],[2,3]) %Stiffness matrix for 2-3 system
u23=K23\F23 %Displacement array for node 2 and 3, same as: inv(K23)*F23

K23 =

   60  -20
  -20   20

u23 =

   0.25000
   0.75000



### Solving the system by solving system of linear equations manually
* Select the 2-3 sub-system: 
$$\begin{Bmatrix} 0 \\ 10 \end{Bmatrix}=\begin{bmatrix} 60 & -20 \\ -20 & 20\end{bmatrix}\begin{Bmatrix} u_2 \\ u_3\end{Bmatrix}$$
* "Take out" two equations:
$$0=(60 u_2)+(-20 u_3)$$
$$10=(-20 u_2)+(20 u_3)$$
* Express $u_2$ in terms of $u_3$ or vice-versa:
$$0=(60 u_2)+(-20 u_3)\rightarrow 20 u_3=60 u_2 \rightarrow u_2=u_3/3  \rightarrow u_3=3u_2 $$
* Substitute into other equation e.g.:
$$10=(-20 [u_3/3])+(20 u_3) \rightarrow 10=(-20/3 u_3)+(20 u_3) \rightarrow 10=(20-20/3) u_3$$
$$\rightarrow u_3=\frac{10}{20-\frac{20}{3}}=0.75$$
$$\rightarrow u_2=u_3/3=0.25$$

### Compute force array F
Since all nodal diplacements are now known the full force array can now be computed from:
$$\begin{Bmatrix} F_1 \\ F_2 \\ F_3 \end{Bmatrix}=\begin{bmatrix} 40 & -40 & 0 \\ -40 & 60 & -20 \\ 0 & -20 & 20\end{bmatrix}\begin{Bmatrix} 0 \\ 0.25 \\ 0.75\end{Bmatrix}=\begin{Bmatrix} -10 \\ 0 \\ 10 \end{Bmatrix}$$

In [10]:
U=[0; u23(1); u23(2)] %Full displacement array
F=K*U %Compute force array

U =

   0.00000
   0.25000
   0.75000

F =

  -1.0000e+01
  -7.1054e-15
   1.0000e+01



### Computing element forces
The element force data can now be computed too from:
$$\begin{Bmatrix} F^{(i)} \end{Bmatrix}=\begin{bmatrix} k^{(i)}\end{bmatrix}\begin{Bmatrix} u^{(i)} \end{Bmatrix}$$   
  
E.g.:   

$$\begin{Bmatrix} f_{1x}^{(1)} \\ f_{2x}^{(1)} \end{Bmatrix}=\frac{A^{(1)}E^{(1)}}{L^{(1)}}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}\begin{Bmatrix} u_1 \\ u_2\end{Bmatrix}=\begin{bmatrix} k_1 & -k_1 \\ -k_1 & k_1\end{bmatrix}\begin{Bmatrix} u_1 \\ u_2\end{Bmatrix}$$
$$\begin{Bmatrix} f_{2x}^{(2)} \\ f_{3x}^{(2)} \end{Bmatrix}=\frac{A^{(2)}E^{(2)}}{L^{(2)}}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}\begin{Bmatrix} u_2 \\ u_3\end{Bmatrix}=\begin{bmatrix} k_2 & -k_2 \\ -k_2 & k_2\end{bmatrix}\begin{Bmatrix} u_2 \\ u_3\end{Bmatrix}$$

In [11]:
f1=K1*U([1,2]) %Element 1 forces
f2=K2*U([2,3]) %Element 2 forces

f1 =

  -10.0000
  10.0000

f2 =

  -10
   10



### Compute strains
* Hooke's law for a truss   
    * Stress: 
    $$\sigma=E\epsilon=\frac{F}{A}$$
    * Strain: 
    $$\epsilon=\frac{(u_j-u_i)}{L_0}$$    
    
$$\epsilon^{(1)}=\frac{(u_2-u_1)}{L^{(1)}}=\frac{(0.25-0)}{100}=0.0025$$    
$$\epsilon^{(2)}=\frac{(u_3-u_2)}{L^{(2)}}=\frac{(0.75-0.25)}{100}=0.005$$ 

# How about tilted trusses? 

![](2Truss_elephant.png)

# The truss element in a GLOBAL coordinate system
The example below shows a single truss, with a force on either end, in a **global** coordinate system. Local quantities will have their usual notation, global quantities will have a superscript $0$. Next we will explore how to convert the local system to the global system. 
![truss](truss2_3D.png)  


## Step1: Express quantities as 2D vectors
![truss](truss2_2D_01.png)  

* Expand scalar forces and displacements to vectors by adding y-components (all zero in local system) 

$$\mathbf{f_{1}}=\frac{AE}{L}(\mathbf{u1}-\mathbf{u2})$$  
$$\mathbf{f_{2}}=\frac{AE}{L}(\mathbf{u2}-\mathbf{u1})$$  

with: $\mathbf{f_{1}}=\begin{bmatrix} f_{1x} \\ f_{1y}\end{bmatrix}=\begin{bmatrix} -T_{x} \\ 0 \end{bmatrix}$, $\mathbf{f_{2}}=\begin{bmatrix} f_{2x} \\ f_{2y}\end{bmatrix}=\begin{bmatrix} T_{x} \\ 0 \end{bmatrix}$, $\mathbf{u_{1}}=\begin{bmatrix} u_{1x} \\ u_{1y}\end{bmatrix}$, $\mathbf{u_{2}}=\begin{bmatrix} u_{2x} \\ u_{2y}\end{bmatrix}$  


* The 1D truss element equation
$$\begin{Bmatrix} f \end{Bmatrix}=\begin{Bmatrix} f_{1} \\ f_{2} \end{Bmatrix}=\frac{AE}{L}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}\begin{Bmatrix} u_1 \\ u_2\end{Bmatrix}$$

, can then be rewritten:
$$\begin{Bmatrix} f \end{Bmatrix}=\begin{Bmatrix} \mathbf{f_{1}} \\ \mathbf{f_{2}} \end{Bmatrix} =\begin{Bmatrix} f_{1x} \\ f_{1y} \\ f_{2x} \\ f_{2y} \end{Bmatrix}=\frac{AE}{L}\begin{bmatrix} 1 & 0  & -1 & 0 \\ 0 & 0 & 0 & 0 \\ -1 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0\end{bmatrix}\begin{Bmatrix} u1_{x} \\ u1_{y} \\ u2_{x} \\ u2_{y} \end{Bmatrix}$$  

* The added zeros stem from the fact that the local y-components are all zero for a truss (1D element)

## Step2: Transformation of the 2D local equations to the 2D global system
### Transforming vector and tensor quantities
* Define a transformation matrix, in this case a rotation tensor ($l$ and $m$ named "direction cosines")
$$\boldsymbol{\xi}=\begin{bmatrix} \cos(\alpha) & -\sin(\alpha) \\ \sin(\alpha) & \cos(\alpha)\end{bmatrix}=\begin{bmatrix} l & -m \\ m & l\end{bmatrix}$$ 

* Allowing one to transform **vector** and **tensor** quantities in the **local** system to the **global** system 
$$\mathbf{f_{i}^0}=\boldsymbol{\xi}\mathbf{f_{i}}$$
$$\mathbf{u_{i}^0}=\boldsymbol{\xi}\mathbf{u_{i}}$$
$$\boldsymbol{\sigma^0}=\boldsymbol{\xi}\boldsymbol{\sigma}\boldsymbol{\xi}^\top$$


* Inverse of $\boldsymbol{\xi}$ which is $\boldsymbol{\xi}^{-1}=\boldsymbol{\xi}^\top$ enables inverse mapping   
$$\mathbf{f_{i}}=\boldsymbol{\xi}^\top \mathbf{f_{i}^0}$$
$$\mathbf{u_{i}}=\boldsymbol{\xi}^\top \mathbf{u_{i}^0}$$
$$\boldsymbol{\sigma}=\boldsymbol{\xi}^\top\boldsymbol{\sigma^0}\boldsymbol{\xi}$$


* Note that the rotation tensor can be chosen to define the local-to-global tranformation OR the global-to-local transformation. This is simly a choice of convention. 
    * If $\boldsymbol{\xi}$ is for a local-to-global tranformation then $\boldsymbol{\xi}^\top$ enables the global-to-local tranformation. 
    * If $\boldsymbol{\xi}$ is for a global-to-local tranformation then $\boldsymbol{\xi}^\top$ enables the local-to-global tranformation instead.  

### Transforming array or matrix quantities
* For the force, displacement **arrays**, and stiffness **matrix** a dedicated transformation matrix $\begin{bmatrix} \lambda \end{bmatrix}$ is required:  
$$\begin{bmatrix} \lambda \end{bmatrix}=\begin{bmatrix} \boldsymbol{\xi} & \mathbf{0} \\ \mathbf{0} & \boldsymbol{\xi} \end{bmatrix}=\begin{bmatrix} \cos(\alpha) & -\sin(\alpha) & 0 & 0 \\ \sin(\alpha) & \cos(\alpha) & 0 & 0 \\ 0 & 0 & \cos(\alpha) & -\sin(\alpha) \\ 0 & 0 & \sin(\alpha) & \cos(\alpha) \end{bmatrix}=\begin{bmatrix} l & -m & 0 & 0 \\ m & l & 0 & 0 \\ 0 & 0 & l & -m \\ 0 & 0 & m & l \end{bmatrix}$$

* Allowing one to transform **matrix** or **array** quantities from the local to the global system
$$\begin{Bmatrix} f^0 \end{Bmatrix}=\begin{bmatrix} \lambda \end{bmatrix} \begin{Bmatrix} f \end{Bmatrix}$$
$$\begin{Bmatrix} u^0 \end{Bmatrix}=\begin{bmatrix} \lambda \end{bmatrix} \begin{Bmatrix} u \end{Bmatrix}$$
$$\begin{bmatrix} K^0 \end{bmatrix}=\begin{bmatrix} \lambda \end{bmatrix} \begin{bmatrix} K \end{bmatrix}\begin{bmatrix} \lambda \end{bmatrix}^\top$$
* The inverse of $\begin{bmatrix} \lambda \end{bmatrix}$ which is $\begin{bmatrix} \lambda \end{bmatrix}^{-1}=\begin{bmatrix} \lambda \end{bmatrix}^\top$ enables the inverse mapping 
$$\begin{Bmatrix} f \end{Bmatrix}=\begin{bmatrix} \lambda \end{bmatrix}^\top \begin{Bmatrix} f^0 \end{Bmatrix}$$
$$\begin{Bmatrix} u \end{Bmatrix}=\begin{bmatrix} \lambda \end{bmatrix}^\top \begin{Bmatrix} u^0 \end{Bmatrix}$$
$$\begin{bmatrix} K \end{bmatrix}=\begin{bmatrix} \lambda \end{bmatrix}^\top \begin{bmatrix} K^0 \end{bmatrix}\begin{bmatrix} \lambda \end{bmatrix}$$

* Which means the element equations can be formulation in the local or global system
$$ \begin{Bmatrix} f \end{Bmatrix} = \begin{bmatrix} K \end{bmatrix} \begin{Bmatrix} u \end{Bmatrix} \rightarrow  \begin{Bmatrix} f^0 \end{Bmatrix} = \begin{bmatrix} K^0 \end{bmatrix} \begin{Bmatrix} u^0 \end{Bmatrix} $$

#### Transforming the force arrays to the global coordinate system
* The element force array in the global system is derived from multiplication with $\begin{bmatrix} \lambda \end{bmatrix}$:   
$$\begin{Bmatrix} f^0 \end{Bmatrix}=\begin{Bmatrix} f^0_{1x} \\ f^0_{1y} \\ f^0_{2x} \\ f^0_{2y} \end{Bmatrix}=\begin{bmatrix} \cos(\alpha) & -\sin(\alpha) & 0 & 0 \\ \sin(\alpha) & \cos(\alpha) & 0 & 0 \\ 0 & 0 & \cos(\alpha) & -\sin(\alpha) \\ 0 & 0 & \sin(\alpha) & \cos(\alpha) \end{bmatrix}\begin{Bmatrix} -T \\ 0 \\ T \\ 0 \end{Bmatrix}=\begin{Bmatrix} -T \cos(\alpha) \\ -T \sin(\alpha) \\ T \cos(\alpha) \\ T \sin(\alpha) \end{Bmatrix}$$   
   
   
* If the "direction cosines" are given the symbols $l=\cos(\alpha)$ and $m=\sin(\alpha)$ this can be written as:   
$$\begin{Bmatrix} f^0 \end{Bmatrix}=\begin{Bmatrix} f^0_{1x} \\ f^0_{1y} \\ f^0_{2x} \\ f^0_{2y} \end{Bmatrix}=\begin{bmatrix} l & -m & 0 & 0 \\ m & l & 0 & 0 \\ 0 & 0 & l & -m \\ 0 & 0 & m & l \end{bmatrix}\begin{Bmatrix} -T \\ 0 \\ T \\ 0 \end{Bmatrix}=\begin{Bmatrix} -T l \\ -T m \\ T l \\ T m \end{Bmatrix} $$   

#### Transforming the displacement arrays to the global coordinate system
* The element displacement array in the global system is derived from multiplication with $\begin{bmatrix} \lambda \end{bmatrix}$:   
$$\begin{Bmatrix} u^0 \end{Bmatrix}=\begin{Bmatrix} u^0_{1x} \\ u^0_{1y} \\ u^0_{2x} \\ u^0_{2y} \end{Bmatrix}=\begin{bmatrix} \cos(\alpha) & -\sin(\alpha) & 0 & 0 \\ \sin(\alpha) & \cos(\alpha) & 0 & 0 \\ 0 & 0 & \cos(\alpha) & -\sin(\alpha) \\ 0 & 0 & \sin(\alpha) & \cos(\alpha) \end{bmatrix}\begin{Bmatrix} u_{1x} \\ 0 \\ u_{2x} \\ 0 \end{Bmatrix}=\begin{Bmatrix} u_{1x} \cos(\alpha) \\ u_{1x} \sin(\alpha) \\ u_{2x} \cos(\alpha) \\ u_{2x} \sin(\alpha) \end{Bmatrix}$$   
   
   
* If the "direction cosines" are given the symbols $l=\cos(\alpha)$ and $m=\sin(\alpha)$ this can be written as:   
$$\begin{Bmatrix} u^0 \end{Bmatrix}=\begin{Bmatrix} u^0_{1x} \\ u^0_{1y} \\ u^0_{2x} \\ u^0_{2y} \end{Bmatrix}=\begin{bmatrix} l & -m & 0 & 0 \\ m & l & 0 & 0 \\ 0 & 0 & l & -m \\ 0 & 0 & m & l \end{bmatrix}\begin{Bmatrix} u_{1x} \\ 0 \\ u_{2x} \\ 0 \end{Bmatrix}=\begin{Bmatrix} u_{1x} l \\ u_{1x} m \\ u_{2x} l \\ u_{2x} m \end{Bmatrix} $$   

#### Transforming the stiffness arrays to the global coordinate system
* The element stiffness array in the global system is derived from pre- and post rotation with $\begin{bmatrix} \lambda \end{bmatrix}$:   
$$\begin{bmatrix} K^0 \end{bmatrix}=\begin{bmatrix} \cos(\alpha) & -\sin(\alpha) & 0 & 0 \\ \sin(\alpha) & \cos(\alpha) & 0 & 0 \\ 0 & 0 & \cos(\alpha) & -\sin(\alpha) \\ 0 & 0 & \sin(\alpha) & \cos(\alpha) \end{bmatrix} \frac{AE}{L}\begin{bmatrix} 1 & 0  & -1 & 0 \\ 0 & 0 & 0 & 0 \\ -1 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0\end{bmatrix} \begin{bmatrix} \cos(\alpha) & \sin(\alpha) & 0 & 0 \\ -\sin(\alpha) & \cos(\alpha) & 0 & 0 \\ 0 & 0 & \cos(\alpha) & \sin(\alpha) \\ 0 & 0 & -\sin(\alpha) & \cos(\alpha) \end{bmatrix}$$   
   
   
* If the "direction cosines" are given the symbols $l=\cos(\alpha)$ and $m=\sin(\alpha)$ this can be written as:   
$$\begin{bmatrix} K^0 \end{bmatrix}=\begin{bmatrix} l & -m & 0 & 0 \\ m & l & 0 & 0 \\ 0 & 0 & l & -m \\ 0 & 0 & m & l \end{bmatrix} \frac{AE}{L}\begin{bmatrix} 1 & 0  & -1 & 0 \\ 0 & 0 & 0 & 0 \\ -1 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0\end{bmatrix} \begin{bmatrix} l & m & 0 & 0 \\ -m & l & 0 & 0 \\ 0 & 0 & l & m \\ 0 & 0 & -m & l \end{bmatrix}= \frac{AE}{L}\begin{bmatrix}l^2 & lm & -l^2 & -lm \\ lm &  m^2 & -lm & -m^2 \\ -l^2 & -lm &  l^2 &  lm \\ -lm & -m^2 &  lm &  m^2 \end{bmatrix}$$   


*Recall the matrix multiplication rule:* $\begin{bmatrix}C\end{bmatrix}=\begin{bmatrix}A\end{bmatrix}\begin{bmatrix}B\end{bmatrix} \rightarrow c_{ij}=a_{ik}b_{kj}$ 



## The truss in the LOCAL system
![truss](truss2_2D_01.png)  

* The 2D truss element equation in the local system
$$\begin{Bmatrix} f \end{Bmatrix}=\begin{Bmatrix} \mathbf{f_{1}} \\ \mathbf{f_{2}} \end{Bmatrix} =\begin{Bmatrix} f_{1x} \\ f_{1y} \\ f_{2x} \\ f_{2y} \end{Bmatrix}=\frac{AE}{L}\begin{bmatrix} 1 & 0  & -1 & 0 \\ 0 & 0 & 0 & 0 \\ -1 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0\end{bmatrix}\begin{Bmatrix} u1_{x} \\ u1_{y} \\ u2_{x} \\ u2_{y} \end{Bmatrix}$$  


# The truss element in a GLOBAL coordinate system
The example below shows a single truss, with a force on either end, in a **global** coordinate system. Local quantities will have their usual notation, global quantities will have a superscript $0$. Next we will explore how to convert the local system to the global system. 
![truss](truss2_3D.png)  

* The truss element equation in the global 2D system
$$\begin{Bmatrix} f^0 \end{Bmatrix}=\begin{Bmatrix} \mathbf{f_{1}^0} \\ \mathbf{f_{2}^0} \end{Bmatrix} =\begin{Bmatrix} f_{1x} l \\ f_{1x} m \\ f_{2x} l \\ f_{2x} m \end{Bmatrix}= \frac{AE}{L}\begin{bmatrix}l^2 & lm & -l^2 & -lm \\ lm &  m^2 & -lm & -m^2 \\ -l^2 & -lm &  l^2 &  lm \\ -lm & -m^2 &  lm &  m^2 \end{bmatrix}\begin{Bmatrix} u_{1x} l \\ u_{1x} m \\ u_{2x} l \\ u_{2x} m \end{Bmatrix}$$  


# Generalisation to 3D
## Step1: Express quantities as 2D vectors
![truss](truss2_3D_01.png)  

* Expand scalar forces and displacements to vectors by adding y-components (all zero in local system) 

$$\mathbf{f_{1}}=\frac{AE}{L}(\mathbf{u1}-\mathbf{u2})$$  
$$\mathbf{f_{2}}=\frac{AE}{L}(\mathbf{u2}-\mathbf{u1})$$  

with: $\mathbf{f_{1}}=\begin{bmatrix} f_{1x} \\ f_{1y} \\ f_{1z}\end{bmatrix}=\begin{bmatrix} -T_{x} \\ 0 \\ 0\end{bmatrix}$, $\mathbf{f_{2}}=\begin{bmatrix} f_{2x} \\ f_{2y} \\ f_{2z} \end{bmatrix}=\begin{bmatrix} T_{x} \\ 0 \\ 0\end{bmatrix}$, $\mathbf{u_{1}}=\begin{bmatrix} u_{1x} \\ u_{1y} \\u_{1z}\end{bmatrix}$, $\mathbf{u_{2}}=\begin{bmatrix} u_{2x} \\ u_{2y} \\ u_{2z}\end{bmatrix}$  


## Step2: Transformation of the 3D local equations to the 3D global system
* So far a single rotation in 2D, around the z-axis, has been shown. 
* A general rotation in 3D can be defined in multiple ways e.g. using so called [Euler angles](https://en.wikipedia.org/wiki/Euler_angles) or instead using "yaw, pitch, and roll" angles (see figure). 
![angles](Yaw_Axis_Corrected.png)  

*See also: [https://en.wikipedia.org/wiki/Euler_angles](https://en.wikipedia.org/wiki/Euler_angles)*

### Define Euler angles and rotation tensors
* A triplet of (Euler) rotation angles ($\alpha$, $\beta$, and $\gamma$) and rotation tensors ($\boldsymbol{\xi_x}$, $\boldsymbol{\xi_y}$, and $\boldsymbol{\xi_z}$) can be created which together define $\boldsymbol{\xi}$: 

$$\boldsymbol{\xi}=\boldsymbol{\xi_x}\boldsymbol{\xi_y}\boldsymbol{\xi_z}$$

$$\boldsymbol{\xi_x}=\begin{bmatrix} 1 & 0 & 0 \\ 0 & \cos(\alpha) & -\sin(\alpha) \\ 0 & \sin(\alpha) & \cos(\alpha) \end{bmatrix}$$  

$$\boldsymbol{\xi_y}=\begin{bmatrix} \cos(\beta) & 0 & \sin(\beta) \\ 0 & 1 &  0 \\ -\sin(\beta) & 0 & \cos(\beta) \end{bmatrix}$$  

$$\boldsymbol{\xi_z}=\begin{bmatrix} \cos(\gamma) & -\sin(\gamma) & 0 \\ \sin(\gamma) & \cos(\gamma) & 0 \\ 0 & 0 & 1\end{bmatrix}$$

*See also: [https://en.wikipedia.org/wiki/Euler_angles](https://en.wikipedia.org/wiki/Euler_angles)*

## Derive global element equations
![truss](truss2_3D_02.png)  

* The truss element equation in the local 3D system
$$\begin{Bmatrix} f \end{Bmatrix}=\begin{Bmatrix} \mathbf{f_{1}} \\ \mathbf{f_{2}} \end{Bmatrix} =\begin{Bmatrix} f_{1x} \\ f_{1y} \\ f_{1z} \\ f_{2x} \\ f_{2y} \\ f_{2z} \end{Bmatrix}=\frac{AE}{L}\begin{bmatrix} 1 & 0  & 0 & -1 & 0 & 0\\ 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 \\ -1 & 0 & 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 \end{bmatrix}\begin{Bmatrix} u1_{x} \\ u1_{y} \\ u1_{z} \\ u2_{x} \\ u2_{y} \\ u2_{z} \end{Bmatrix}$$  

### Transforming array or matrix quantities
* For the force, displacement **arrays**, and stiffness **matrix** a dedicated transformation matrix $\begin{bmatrix} \lambda \end{bmatrix}$ is required:  

$$\begin{bmatrix} \lambda \end{bmatrix}=\begin{bmatrix} \boldsymbol{\xi} & \mathbf{0} \\ \mathbf{0} & \boldsymbol{\xi} \end{bmatrix}$$

* Note that in the above each entry is a 3x3 tensor, the whole matrix is therefore a 6x6 matrix

# Example: A 2 truss structure in 2D
Given the two truss problem below, provide (in the global system) the element equations, the global stiffness matrix, and the finite element equations. 

![truss_3D](example_3D_truss.png)



### Summarise the known quantities and relationships

$$k^{(1)}=\frac{A^{(1)}E^{(1)}}{L^{(1)}}=\frac{50*200*10^3}{1000}=10000$$
$$k^{(2)}=\frac{A^{(2)}E^{(2)}}{L^{(2)}}=\frac{40*200*10^3}{800}=10000$$


## Step 1: Express quantities as 2D vectors and rewrite element equations accordingly

$$\begin{Bmatrix} f^{(i)} \end{Bmatrix}=\begin{Bmatrix} \mathbf{f_{1}}^{(i)} \\ \mathbf{f_{2}}^{(i)} \end{Bmatrix} =\begin{Bmatrix} f_{1x}^{(i)} \\ f_{1y}^{(i)} \\ f_{2x}^{(i)} \\ f_{2y}^{(i)} \end{Bmatrix}=k^{(i)}\begin{bmatrix} 1 & 0  & -1 & 0 \\ 0 & 0 & 0 & 0 \\ -1 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0\end{bmatrix}\begin{Bmatrix} u1_{x}^{(i)} \\ u1_{y}^{(i)} \\ u2_{x}^{(i)} \\ u2_{y}^{(i)} \end{Bmatrix}$$ 

## Step2: Transformation of the 2D local equations to the 2D global system
### Transforming vector and tensor quantities
* Define a transformation matrix, in this case a rotation tensor ($l$ and $m$ named "direction cosines")
$$\boldsymbol{\xi}=\begin{bmatrix} \cos(\alpha) & -\sin(\alpha) \\ \sin(\alpha) & \cos(\alpha)\end{bmatrix}=\begin{bmatrix} l & -m \\ m & l\end{bmatrix}$$ 

* With:
$$l=\frac{800}{1000}=0.8$$
$$m=\frac{-600}{1000}=-0.6$$

* The transformation matrix $\begin{bmatrix} \lambda \end{bmatrix}$ 

$$\begin{bmatrix} \lambda \end{bmatrix}=\begin{bmatrix} \boldsymbol{\xi} & \mathbf{0} \\ \mathbf{0} & \boldsymbol{\xi} \end{bmatrix}=\begin{bmatrix} \cos(\alpha) & -\sin(\alpha) & 0 & 0 \\ \sin(\alpha) & \cos(\alpha) & 0 & 0 \\ 0 & 0 & \cos(\alpha) & -\sin(\alpha) \\ 0 & 0 & \sin(\alpha) & \cos(\alpha) \end{bmatrix}=\begin{bmatrix} l & -m & 0 & 0 \\ m & l & 0 & 0 \\ 0 & 0 & l & -m \\ 0 & 0 & m & l \end{bmatrix}$$

* For element 1 we have: 
    * The force vector in the global system
$$\begin{Bmatrix} f^{(1)} \end{Bmatrix}^{0}=\begin{bmatrix} \lambda \end{bmatrix}\begin{Bmatrix} f^{(1)} \end{Bmatrix}=\begin{bmatrix} l & -m & 0 & 0 \\ m & l & 0 & 0 \\ 0 & 0 & l & -m \\ 0 & 0 & m & l \end{bmatrix}\begin{Bmatrix} f_{1x}^{(1)} \\ 0 \\ f_{2x}^{(1)} \\ 0 \end{Bmatrix}=\begin{Bmatrix} f_{1x}^{(1)} l \\ f_{1x}^{(1)} m \\ f_{2x}^{(1)} l \\ f_{2x}^{(1)} m \end{Bmatrix}$$
    * The displacement vector in the global system
$$\begin{Bmatrix} u^{(1)} \end{Bmatrix}^{0}=\begin{bmatrix} \lambda \end{bmatrix}\begin{Bmatrix} u^{(1)} \end{Bmatrix}=\begin{bmatrix} l & -m & 0 & 0 \\ m & l & 0 & 0 \\ 0 & 0 & l & -m \\ 0 & 0 & m & l \end{bmatrix}\begin{Bmatrix} u_{1x}^{(1)} \\ 0 \\ u_{2x}^{(1)} \\ 0 \end{Bmatrix}=\begin{Bmatrix} u_{1x}^{(1)} l \\ u_{1x}^{(1)} m \\ u_{2x}^{(1)} l \\ u_{2x}^{(1)} m \end{Bmatrix}$$

* For element 1 we have: 
    * The stiffness matrix in the global system    
$$\begin{bmatrix} K^0 \end{bmatrix}^{1}=\begin{bmatrix} l & -m & 0 & 0 \\ m & l & 0 & 0 \\ 0 & 0 & l & -m \\ 0 & 0 & m & l \end{bmatrix} k^{(1)}\begin{bmatrix} 1 & 0  & -1 & 0 \\ 0 & 0 & 0 & 0 \\ -1 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0\end{bmatrix} \begin{bmatrix} l & -m & 0 & 0 \\ m & l & 0 & 0 \\ 0 & 0 & l & -m \\ 0 & 0 & m & l \end{bmatrix}= k^{(1)}\begin{bmatrix}l^2 & lm & -l^2 & -lm \\ lm &  m^2 & -lm & -m^2 \\ -l^2 & -lm &  l^2 &  lm \\ -lm & -m^2 &  lm &  m^2 \end{bmatrix}$$   
    * Leading to the element equations in the global sytem
$$\begin{Bmatrix} f_{1x}^{(1)} l \\ f_{1x}^{(1)} m \\ f_{2x}^{(1)} l \\ f_{2x}^{(1)} m \end{Bmatrix}=k^{(1)}\begin{bmatrix}l^2 & lm & -l^2 & -lm \\ lm &  m^2 & -lm & -m^2 \\ -l^2 & -lm &  l^2 &  lm \\ -lm & -m^2 &  lm &  m^2 \end{bmatrix}\begin{Bmatrix} u_{1x}^{(1)} l \\ u_{1x}^{(1)} m \\ u_{2x}^{(1)} l \\ u_{2x}^{(1)} m \end{Bmatrix}$$  


* For element 2 we have
    * Since the local and global systems coincide: 
    $$\begin{Bmatrix} f^{(2)} \end{Bmatrix}^{0}=\begin{Bmatrix} f^{(2)} \end{Bmatrix}$$ 
    $$\begin{Bmatrix} u^{(2)} \end{Bmatrix}^{0}=\begin{Bmatrix} u^{(2)} \end{Bmatrix}$$
    $$\begin{bmatrix} K^{(2)} \end{bmatrix}^{0}=\begin{bmatrix} K^{(2)} \end{bmatrix}$$
    
    * Leading to the element equations in the global sytem:
$$\begin{Bmatrix} f_{1x}^{(2)} \\ f_{1y}^{(2)} \\ f_{2x}^{(2)} \\ f_{2y}^{(2)} \end{Bmatrix}=k^{(2)}\begin{bmatrix} 1 & 0  & -1 & 0 \\ 0 & 0 & 0 & 0 \\ -1 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0\end{bmatrix}\begin{Bmatrix} u1_{x}^{(2)} \\ u1_{y}^{(2)} \\ u2_{x}^{(2)} \\ u2_{y}^{(2)} \end{Bmatrix}$$  


## Super-position to formulate the global stiffness matrix
Super-position still "involves" nodal indices. However now there are 2 components (e.g. x- and y-components) at each node. Therefore the global stiffness becomes a $(2*N)\times(2*N)$ matrix (rather than an $N \times N$ matrix). However you can identify an $N \times N$ tiling of 2x2 groups which contain data for a particular node. 

$$\begin{bmatrix}K^0 \end{bmatrix}=\begin{bmatrix}k^{(1)}l^2 & k^{(1)}lm & -k^{(1)}l^2 & -k^{(1)}lm & 0 & 0\\ k^{(1)}lm &  k^{(1)}m^2 & -k^{(1)}lm & -k^{(1)}m^2 & 0 & 0\\ -k^{(1)}l^2 & -k^{(1)}lm &  k^{(1)}l^2+k^{(2)} &  k^{(1)}lm & -k^{(2)} & 0\\ -k^{(1)}lm & -k^{(1)}m^2 &  k^{(1)}lm &  k^{(1)}m^2 & 0 & 0 \\ 0 & 0 & -k^{(2)} & 0 & k^{(2)} & 0 \\ 0 & 0 & 0 & 0 & 0 & 0\end{bmatrix}$$
$$\begin{bmatrix}K^0 \end{bmatrix}=\begin{bmatrix}   6400 &   4800 &   -6400 &  -4800 &       0 &  0 & \\ 4800 &   3600 &   -4800 &  -3600 &       0 &  0 & \\ -6400 &  -4800 &   16400 &   4800 &  -10000 &  0 & \\ -4800 &  -3600 &    4800 &   3600 &       0 &  0 & \\ 0 &      0 &  -10000 &      0 &   10000 &  0 & \\ 0 &      0 &       0 &      0 &       0 &  0 & \\\end{bmatrix}$$



### Numerical example
#### Element stiffness matrix for element 1

In [2]:
% Set up element 1 components
E1=200000; %Youngs modulus
L1=1000; %Length
A1=50; %Area
k1=(A1*E1)/L1; %k "stiffness" value

l=800/1000; %cos(alpha)
m=-600/1000; %sin(alpha)
X=[l -m; m l]; %Xi rotation tensor
L=[l -m 0 0; m l 0 0; 0 0 l -m; 0 0 m l]; # Lambda matrix

Q=[1 0 -1 0;0 0 0 0; -1 0 1 0; 0 0 0 0]; %K/k
K1=k1*Q %Element stiffness matrix in local system
K1_0=L*K1*L' %Element stiffness matrix in global system

K1 =

   10000       0  -10000       0
       0       0       0       0
  -10000       0   10000       0
       0       0       0       0

K1_0 =

   6400  -4800  -6400   4800
  -4800   3600   4800  -3600
  -6400   4800   6400  -4800
   4800  -3600  -4800   3600



#### Element stiffness matrix for element 2

In [3]:
% Set up element 2 components
E2=200000; %Youngs modulus
L2=800; %Length
A2=40; %Area
Q=[1 0 -1 0;0 0 0 0; -1 0 1 0; 0 0 0 0]; %K/k
k2=(A2*E2)/L2; %k "stiffness" value
K2=k2*Q %Element stiffness matrix in local system
K2_0=K2 %Element stiffness matrix in global system

K2 =

   10000       0  -10000       0
       0       0       0       0
  -10000       0   10000       0
       0       0       0       0

K2_0 =

   10000       0  -10000       0
       0       0       0       0
  -10000       0   10000       0
       0       0       0       0



#### Global stiffness matrix assembly

In [14]:
% Super-position to get global stiffness matrix
K_0=zeros(6,6); %Initialize
K_0(1:4,1:4)=K1_0; %Add part 1
K_0(3:6,3:6)=K_0(3:6,3:6)+K2_0 %Add part 2

K_0 =

    6400   -4800   -6400    4800       0       0
   -4800    3600    4800   -3600       0       0
   -6400    4800   16400   -4800  -10000       0
    4800   -3600   -4800    3600       0       0
       0       0  -10000       0   10000       0
       0       0       0       0       0       0



### The global force and displacement arrays
* The global force and displacement arrays are:    
$\begin{Bmatrix} F^0 \end{Bmatrix}=\begin{Bmatrix} F_{1x}^0 \\ F_{1y}^0 \\ F_{2x}^0 \\  F_{2y}^0 \\  F_{3x}^0 \\  F_{3y}^0 \end{Bmatrix}$, and $\begin{Bmatrix} u^0 \end{Bmatrix}=\begin{Bmatrix} u_{1x}^0 \\ u_{1y}^0 \\ u_{2x}^0 \\  u_{2y}^0 \\  u_{3x}^0 \\  u_{3y}^0 \end{Bmatrix}$

* Considering the boundary conditions $u_{1x}=u_{1y}=u_{3x}=u_{3y}=0$, and $F_{2x}=0$, and $F_{2y}=-1000$, we get:
$$\begin{Bmatrix} F_{1x}^0 \\ F_{1y}^0 \\ 0 \\  -1000 \\  F_{3x}^0 \\  F_{3y}^0 \end{Bmatrix}=\begin{bmatrix} 6400 &  -4800 &   -6400 &   4800 &       0 &  0 & \\ -4800 &   3600 &    4800 &  -3600 &       0 &  0 & \\ 
  -6400 &   4800 &   16400 &  -4800 &  -10000 &  0 & \\ 4800 &  -3600 &   -4800 &   3600 &       0 &  0 & \\ 
      0 &      0 &  -10000 &      0 &   10000 &  0 & \\ 0 &      0 &       0 &      0 &       0 &  0 & \\\end{bmatrix} \begin{Bmatrix} 0 \\ 0 \\ u_{2x}^0 \\  u_{2y}^0 \\  0 \\  0 \end{Bmatrix}$$

### Solving the system 
#### Computing unknown displacements
* Identify subset defining two equations with two unknowns: 
$$F_{2x}^0 = 16400  u_{2x}^0 - 4800 u_{2y}^0 = 0 $$
$$F_{2y}^0 =  -4800  u_{2x}^0 + 3600 u_{2y}^0 = -1000 $$

* Express $u_{2x}^0$ in terms of $u_{2y}^0$:
$$\rightarrow u_{2x}^0=\frac{4800}{16400} u_{2y}^0=\frac{12}{41} u_{2y}^0$$

* Substitute into other equation:
$$\rightarrow -4800\big(\frac{12}{41} u_{2y}^0\big)+3600u_{2y}^0=\frac{90000}{41} u_{2y}^0=-1000 \rightarrow u_{2y}^0=-\frac{41}{90} \approx -0.4556 $$
$$16400  u_{2x}^0 - 4800 u_{2y}^0 = 0 \rightarrow 16400  u_{2x}^0 - 4800 (-\frac{41}{90}) = 0 $$
$$\rightarrow 16400  u_{2x}^0 +\frac{6560}{3} = 0 \rightarrow 16400  u_{2x}^0 = -\frac{6560}{3} \rightarrow u_{2x}^0 = -\frac{2}{15} \approx -0.1333$$

#### Computing unknown forces
$$\begin{Bmatrix} F_{1x}^0 \\ F_{1y}^0 \\ 0 \\  -1000 \\  F_{3x}^0 \\  F_{3y}^0 \end{Bmatrix}=\begin{bmatrix} 6400 &  -4800 &   -6400 &   4800 &       0 &  0 & \\ -4800 &   3600 &    4800 &  -3600 &       0 &  0 & \\ 
  -6400 &   4800 &   16400 &  -4800 &  -10000 &  0 & \\ 4800 &  -3600 &   -4800 &   3600 &       0 &  0 & \\ 
      0 &      0 &  -10000 &      0 &   10000 &  0 & \\ 0 &      0 &       0 &      0 &       0 &  0 & \\\end{bmatrix} =\begin{Bmatrix} 1333 \frac{1}{3} \\ 1000 \\ 0 \\  -1000 \\  -1333 \frac{1}{3} \\  0 \end{Bmatrix}$$



In [15]:
u=[0 0 -2/15 -41/90 0 0]';
F=K_0*u

F =

  -1333.33333
   1000.00000
      0.00000
  -1000.00000
   1333.33333
      0.00000



# Example: A 2 truss structure in 2D
Consider the two truss system of Figure Q4. Both trusses are linear elastic. Node 1 and node 3 are constrained from moving. The $x^0$ (horizontal) and $y^0$ (vertical) axis directions define the global axis system. A brontosaurus is attached via a rope to node 2, and pulls at it with a force of $P=1.5$ MN (mega Newton) in the x-direction. The two trusses have the following properties: 

Areas: $A^{(1)}=A^{(2)}=0.02$ $m^2$  
Young's moduli: $E^{(1)}=E^{(2}=2$ GPa  
Lengths: $L^{(1)}=L^{(2}=4$ m

__(a)__	Write down the element equations for element 1 and 2 in their local coordinate systems  __[5]__

__(b)__ Transform the local stiffness matrices to the global coordinate system and derive the global stiffness matrix __[10]__

__(c)__ Formulate the global finite element equations of the total system, and solve for the x-, and y-displacement components of node 2. __[15]__ 


![](2Truss_tilted_bronto.png)

## Solution a)

$$\begin{Bmatrix} f \end{Bmatrix}^{(1)}=\begin{Bmatrix} \mathbf{f_{1}} \\ \mathbf{f_{2}} \end{Bmatrix} =\begin{Bmatrix} f_{1x} \\ f_{1y} \\ f_{2x} \\ f_{2y} \end{Bmatrix}=\frac{A_1E_1}{L_1}\begin{bmatrix} 1 & 0  & -1 & 0 \\ 0 & 0 & 0 & 0 \\ -1 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0\end{bmatrix}\begin{Bmatrix} u1_{x} \\ u1_{y} \\ u2_{x} \\ u2_{y} \end{Bmatrix}$$  

$$\begin{Bmatrix} f \end{Bmatrix}^{(1)}=\begin{Bmatrix} \mathbf{f_{1}} \\ \mathbf{f_{2}} \end{Bmatrix} =\begin{Bmatrix} f_{1x} \\ f_{1y} \\ f_{2x} \\ f_{2y} \end{Bmatrix}=10000000 \begin{bmatrix} 1 & 0  & -1 & 0 \\ 0 & 0 & 0 & 0 \\ -1 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0\end{bmatrix}\begin{Bmatrix} u1_{x} \\ u1_{y} \\ u2_{x} \\ u2_{y} \end{Bmatrix}$$  



$$\begin{Bmatrix} f \end{Bmatrix}^{(2)}=\begin{Bmatrix} \mathbf{f_{2}} \\ \mathbf{f_{3}} \end{Bmatrix} =\begin{Bmatrix} f_{2x} \\ f_{2y} \\ f_{3x} \\ f_{3y} \end{Bmatrix}=\frac{A_2E_2}{L_2}\begin{bmatrix} 1 & 0  & -1 & 0 \\ 0 & 0 & 0 & 0 \\ -1 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0\end{bmatrix}\begin{Bmatrix} u2_{x} \\ u2_{y} \\ u3_{x} \\ u3_{y} \end{Bmatrix}$$  

$$\begin{Bmatrix} f \end{Bmatrix}^{(2)}=\begin{Bmatrix} \mathbf{f_{2}} \\ \mathbf{f_{3}} \end{Bmatrix} =\begin{Bmatrix} f_{2x} \\ f_{2y} \\ f_{3x} \\ f_{3y} \end{Bmatrix}=10000000 \begin{bmatrix} 1 & 0  & -1 & 0 \\ 0 & 0 & 0 & 0 \\ -1 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0\end{bmatrix}\begin{Bmatrix} u2_{x} \\ u2_{y} \\ u3_{x} \\ u3_{y} \end{Bmatrix}$$  

## Solution b)

The rotation matrix $\begin{bmatrix} \lambda \end{bmatrix}$ is composed as: 

$$\begin{bmatrix} \lambda \end{bmatrix}=\begin{bmatrix} \boldsymbol{\xi} & \mathbf{0} \\ \mathbf{0} & \boldsymbol{\xi} \end{bmatrix}=\begin{bmatrix} \cos(\alpha) & -\sin(\alpha) & 0 & 0 \\ \sin(\alpha) & \cos(\alpha) & 0 & 0 \\ 0 & 0 & \cos(\alpha) & -\sin(\alpha) \\ 0 & 0 & \sin(\alpha) & \cos(\alpha) \end{bmatrix}=\begin{bmatrix} l & -m & 0 & 0 \\ m & l & 0 & 0 \\ 0 & 0 & l & -m \\ 0 & 0 & m & l \end{bmatrix}=\begin{bmatrix} l & -m & 0 & 0 \\ m & l & 0 & 0 \\ 0 & 0 & l & -m \\ 0 & 0 & m & l \end{bmatrix}$$


With $l^{(1)}=m^{(1)}=\frac{1}{\sqrt{2}}\approx0.70711$ and $l^{(2)}=-m^{(2)}=\frac{1}{\sqrt{2}}\approx0.70711$   
Note that $\big(\frac{1}{\sqrt{2}}\big)^2=0.5$



Leading to: 
$$\begin{bmatrix} K^0 \end{bmatrix}=\begin{bmatrix} \lambda \end{bmatrix} \begin{bmatrix} K \end{bmatrix}\begin{bmatrix} \lambda \end{bmatrix}^\top$$
and 
$$\begin{bmatrix} K^0 \end{bmatrix}^{(i)}=\begin{bmatrix} l & -m & 0 & 0 \\ m & l & 0 & 0 \\ 0 & 0 & l & -m \\ 0 & 0 & m & l \end{bmatrix} \frac{A^{(i)}E^{(i)}}{L^{(i)}}\begin{bmatrix} 1 & 0  & -1 & 0 \\ 0 & 0 & 0 & 0 \\ -1 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0\end{bmatrix} \begin{bmatrix} l & m & 0 & 0 \\ -m & l & 0 & 0 \\ 0 & 0 & l & m \\ 0 & 0 & -m & l \end{bmatrix}= \frac{A^{(i)}E^{(i)}}{L^{(i)}}\begin{bmatrix}l^2 & lm & -l^2 & -lm \\ lm &  m^2 & -lm & -m^2 \\ -l^2 & -lm &  l^2 &  lm \\ -lm & -m^2 &  lm &  m^2 \end{bmatrix}$$   


$$\begin{bmatrix} K^0 \end{bmatrix}^{(1)}=5000000 \begin{bmatrix}1 & 1 & -1 & -1 \\ 1 &  1 & -1 & -1 \\ -1 & -1 &  1 &  1 \\ -1 & -1 &  1 &  1 \end{bmatrix}$$   

$$\begin{bmatrix} K^0 \end{bmatrix}^{(2)}=5000000 \begin{bmatrix}1 & -1 & -1 & 1 \\ -1 &  1 & 1 & -1 \\ -1 & 1 &  1 &  -1 \\ 1 & -1 &  -1 &  1 \end{bmatrix}$$   

$$\begin{bmatrix} K^0 \end{bmatrix}=5000000 \begin{bmatrix}
 1 &  1 & -1 & -1 &  0  &  0 \\ 
 1 &  1 & -1 & -1 &  0  &  0 \\
-1 & -1 &  2 &  0 & -1  &  1 \\
-1 & -1 &  0 &  2 &  1  & -1 \\
 0 &  0 & -1 &  1 &  1  & -1 \\
 0 &  0 &  1 & -1 & -1  &  1 \end{bmatrix}$$   

In [16]:
% Numerical check

E=2e9; A=0.02; L=4; 
k=A*E/L
P=1.5e6; 

Q=[1 0 -1 0;0 0 0 0; -1 0 1 0; 0 0 0 0]; %K/k
K1=k*Q %Element stiffness matrix in local system
K2=K1 %Element stiffness matrix in local system

k =  10000000
K1 =

   10000000          0  -10000000          0
          0          0          0          0
  -10000000          0   10000000          0
          0          0          0          0

K2 =

   10000000          0  -10000000          0
          0          0          0          0
  -10000000          0   10000000          0
          0          0          0          0



In [17]:
% Numerical check

l=1/sqrt(2); %cos(alpha)
m=1/sqrt(2); %sin(alpha)
X=[l -m; m l]; %Xi rotation tensor
Lambda_rot=[l -m 0 0; m l 0 0; 0 0 l -m; 0 0 m l]; # Lambda matrix
K1_0=Lambda_rot*K1*Lambda_rot' %Element stiffness matrix in global system

l=1/sqrt(2); %cos(alpha)
m=-1/sqrt(2); %sin(alpha)
X=[l -m; m l]; %Xi rotation tensor
Lambda_rot=[l -m 0 0; m l 0 0; 0 0 l -m; 0 0 m l]; # Lambda matrix
K2_0=Lambda_rot*K2*Lambda_rot' %Element stiffness matrix in global system

% Super-position to get global stiffness matrix
K_0=zeros(6,6); %Initialize
K_0(1:4,1:4)=K1_0; %Add part 1
K_0(3:6,3:6)=K_0(3:6,3:6)+K2_0; %Add part 2

K_0

K1_0 =

   5000000.00000   5000000.00000  -5000000.00000  -5000000.00000
   5000000.00000   5000000.00000  -5000000.00000  -5000000.00000
  -5000000.00000  -5000000.00000   5000000.00000   5000000.00000
  -5000000.00000  -5000000.00000   5000000.00000   5000000.00000

K2_0 =

   5000000.00000  -5000000.00000  -5000000.00000   5000000.00000
  -5000000.00000   5000000.00000   5000000.00000  -5000000.00000
  -5000000.00000   5000000.00000   5000000.00000  -5000000.00000
   5000000.00000  -5000000.00000  -5000000.00000   5000000.00000

K_0 =

 Columns 1 through 4:

    5000000.00000    5000000.00000   -5000000.00000   -5000000.00000
    5000000.00000    5000000.00000   -5000000.00000   -5000000.00000
   -5000000.00000   -5000000.00000   10000000.00000          0.00000
   -5000000.00000   -5000000.00000          0.00000   10000000.00000
          0.00000          0.00000   -5000000.00000    5000000.00000
          0.00000          0.00000    5000000.00000   -5000000.00000

 Columns 5 and 6:

## Solution c)

$$\begin{Bmatrix} F \end{Bmatrix}=\begin{bmatrix} K \end{bmatrix}\begin{Bmatrix} u \end{Bmatrix}$$

$$\begin{Bmatrix} F1_x \\ F1_y \\ F2_x \\ F2_y \\ F3_x \\ F3_y \end{Bmatrix}=5000000 \begin{bmatrix}
 1 &  1 & -1 & -1 &  0  &  0 \\ 
 1 &  1 & -1 & -1 &  0  &  0 \\
-1 & -1 &  2 &  0 & -1  &  1 \\
-1 & -1 &  0 &  2 &  1  & -1 \\
 0 &  0 & -1 &  1 &  1  & -1 \\
 0 &  0 &  1 & -1 & -1  &  1 \end{bmatrix}\begin{Bmatrix} u1_x \\ u1_y \\ u2_x \\ u2_y \\ u3_x \\ u3_y\end{Bmatrix}$$  

$$\begin{Bmatrix} F \end{Bmatrix}=\begin{bmatrix} K \end{bmatrix}\begin{Bmatrix} u \end{Bmatrix}$$

$$\begin{Bmatrix} F1_x \\ F1_y \\ 1.5\cdot10^6 \\ 0 \\ F3_x \\ F3_y \end{Bmatrix}=5000000 \begin{bmatrix}
 1 &  1 & -1 & -1 &  0  &  0 \\ 
 1 &  1 & -1 & -1 &  0  &  0 \\
-1 & -1 &  2 &  0 & -1  &  1 \\
-1 & -1 &  0 &  2 &  1  & -1 \\
 0 &  0 & -1 &  1 &  1  & -1 \\
 0 &  0 &  1 & -1 & -1  &  1 \end{bmatrix}\begin{Bmatrix} 0 \\ 0 \\ u2_x \\ 0 \\ 0 \\ 0\end{Bmatrix}$$  
 

$$u2_x=\frac{P}{K_{33}}=\frac{1.5\cdot10^6}{10000000}=0.15$$

In [18]:
% Numerical check

u2x=P./K_0(3,3)

u=[0 0 u2x 0 0  0 ]'
F=K_0*u

u2x =  0.15000
u =

   0.00000
   0.00000
   0.15000
   0.00000
   0.00000
   0.00000

F =

   -750000
   -750000
   1500000
         0
   -750000
    750000



\newpage

**About this document**    

This document was created using a [Jupyter notebook](https://jupyter.org/) which allows for the presentation of theory and equations, as well as live (running code) numerical implementations. 

This Jupyter notebook is available [open source](https://github.com/Kevin-Mattheus-Moerman/NUIG_BME_402_6101) and features the [Octave](https://www.gnu.org/software/octave/index) programming language (an open source alternative to MATLAB). If you are interested in running this Jupyter notebook yourself [download and install Octave](https://www.gnu.org/software/octave/download) and install [the Jupyter environment](https://jupyter.org/install). Once both Jupyter and Octave are installed follow [these instructions](https://github.com/calysto/octave_kernel) to configure the use of Octave with Jupyter notebooks. 

To run Jupyter call `jupyter notebook` from your Terminal/Command Prompt. 